## Real experiments for head movements

In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

In [34]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score,f1_score
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC,LinearSVC
from sklearn.dummy import DummyClassifier
from seqlearn.perceptron import StructuredPerceptron
import sklearn_crfsuite
from sklearn_crfsuite import metrics as skcrfmetrics
from scipy.stats import pearsonr,spearmanr

In [35]:
import glob

In [36]:
windowsize="9-11-13"
memory="middle"

PATH = "C:\\Users\\zgk261\\nomco\\"+windowsize+"\\"+memory+"\\"
PATH

'C:\\Users\\zgk261\\nomco\\9-11-13\\middle\\'

In [37]:
!dir /B C:\Users\zgk261\nomco\9-11-13\middle\

F2_M4-all-final2_primary_F2.T-Tilt-ST-SideTurn-HO-HeadOther-S-Shake-W-Waggle-N-Nod-J-Jerk-HB-HeadBackward-HF-HeadForward--rdXY.svm.9-11-13-middle.tab
F2_M4-all-final2_primary_M4.T-Tilt-ST-SideTurn-HO-HeadOther-S-Shake-W-Waggle-N-Nod-J-Jerk-HB-HeadBackward-HF-HeadForward--rdXY.svm.9-11-13-middle.tab
F3_F6-all-final2_primary_F3.T-Tilt-ST-SideTurn-HO-HeadOther-S-Shake-W-Waggle-N-Nod-J-Jerk-HB-HeadBackward-HF-HeadForward--rdXY.svm.9-11-13-middle.tab
F3_F6-all-final2_primary_F6.T-Tilt-ST-SideTurn-HO-HeadOther-S-Shake-W-Waggle-N-Nod-J-Jerk-HB-HeadBackward-HF-HeadForward--rdXY.svm.9-11-13-middle.tab
F4_F1-all-final2_primary_F1.T-Tilt-ST-SideTurn-HO-HeadOther-S-Shake-W-Waggle-N-Nod-J-Jerk-HB-HeadBackward-HF-HeadForward--rdXY.svm.9-11-13-middle.tab
F4_F1-all-final2_primary_F4.T-Tilt-ST-SideTurn-HO-HeadOther-S-Shake-W-Waggle-N-Nod-J-Jerk-HB-HeadBackward-HF-HeadForward--rdXY.svm.9-11-13-middle.tab
F5_F2-all-final2_primary_F2.T-Tilt-ST-SideTurn-HO-HeadOther-S-Shake-W-Waggle-N-Nod-J-Jerk-HB-HeadBac

### SKELETON TO READ FILES, SPEAKERS and PRIMARY SPEAKERS


In [38]:
speaker_set = set()

for each_file in glob.glob(PATH+"*.tab"):
    filename=each_file.split("\\")[-1]
    fileinfo=filename.split("-")
    
    primary_speaker = fileinfo[2].split("_")[-1][:-2] #fileinfo[2] = final2_primary_M1.T
    speakers = fileinfo[0]
    l_speaker,r_speaker = speakers.split("_")

    speaker_set.add(primary_speaker)

In [39]:
speaker_set

{'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6'}

### SKELETON TO READ FILES AND COSS_VALIDATE

In the following way: In each iteration, we will leave one speaker out (two files), train a model and test on those two files.

In [40]:

for speaker_on_focus in speaker_set:
    train_files = []
    test_files  = []
    
    for each_file in glob.glob(PATH+"*.tab"):
        filename=each_file.split("\\")[-1]
        fileinfo=filename.split("-")

        primary_speaker = fileinfo[2].split("_")[-1][:-2] #fileinfo[2] = final2_primary_M1.T
        speakers = fileinfo[0]
        l_speaker,r_speaker = speakers.split("_")
        if primary_speaker == l_speaker:
            secondary_speaker = r_speaker
        else:
            secondary_speaker = l_speaker

        #print (primary_speaker,l_speaker,r_speaker,speaker_on_focus)

        file_df = pd.read_csv(each_file,sep="\t")
        file_df.columns = [el.replace(primary_speaker, "PRIMA").replace(secondary_speaker, "SECON") for el in file_df.columns]

        if primary_speaker != speaker_on_focus:
            #FILES TO TRAIN!
            train_files.append(file_df)
        else:
            #FILES TO TEST!
            test_files.append(file_df)
            
        
        
    training_data = pd.concat(train_files, ignore_index=True)
    testing_data = pd.concat(test_files, ignore_index=True)
    print ("Speaker on focus:",speaker_on_focus)
    print ("Training data size:",training_data.shape)
    print ("Testing data size:",testing_data.shape)
    print ()


Speaker on focus: F5
Training data size: (181647, 28)
Testing data size: (16413, 28)

Speaker on focus: F6
Training data size: (182822, 28)
Testing data size: (15238, 28)

Speaker on focus: M2
Training data size: (182715, 28)
Testing data size: (15345, 28)

Speaker on focus: M4
Training data size: (181548, 28)
Testing data size: (16512, 28)

Speaker on focus: F1
Training data size: (181919, 28)
Testing data size: (16141, 28)

Speaker on focus: F3
Training data size: (181651, 28)
Testing data size: (16409, 28)

Speaker on focus: M3
Training data size: (181426, 28)
Testing data size: (16634, 28)

Speaker on focus: M5
Training data size: (180649, 28)
Testing data size: (17411, 28)

Speaker on focus: M1
Training data size: (180025, 28)
Testing data size: (18035, 28)

Speaker on focus: F2
Training data size: (181593, 28)
Testing data size: (16467, 28)

Speaker on focus: F4
Training data size: (181137, 28)
Testing data size: (16923, 28)

Speaker on focus: M6
Training data size: (181528, 28)


### Experiments:

It seems that in the 3-6-9 experiments considering both the past and future of frames, the MLP was not the best model, but the Logistic Regression. I think, though, that the MLP will show good performance when cross-validating with the whole data.

I will, then, try 3 classifiers:

  * SVM, because it is the default classifier that is commonly used
  * Logistic Regression, because it seems to perform the best in our initial experiments with two dialogue data
  * MLP, because when including more data, I expect to estimate a more general (and thus, better) model

In [41]:
accuracy_results = pd.DataFrame(0,columns=speaker_set,index=["MF"])
accuracy_results

,F5,F6,M2,M4,F1,F3,M3,M5,M1,F2,F4,M6
MF,0,0,0,0,0,0,0,0,0,0,0,0


In [42]:
import math

def map_movements(mov_class):
    if mov_class == "Nod" or mov_class == "Jerk":
        return 1
    elif mov_class == "Shake":
        return 2
    elif mov_class!=mov_class: #This returns True if the value is NAN, and it works. It might be source of errors.
        return 0
    else:
        return 3



In [43]:
accuracy_results = pd.DataFrame(0,columns=speaker_set,index=["MF"])
f1score_results = pd.DataFrame(0,columns=speaker_set,index=["MF"])
f1score_resultsw = pd.DataFrame(0,columns=speaker_set,index=["MF"])
#f1score_resultsb = pd.DataFrame(0,columns=speaker_set,index=["MF"])
#f1_score(y_true, y_pred, average='micro')

for speaker_on_focus in speaker_set:
    train_files = []
    test_files  = []
    
    for each_file in glob.glob(PATH+"*.tab"):
        filename=each_file.split("\\")[-1]
        fileinfo=filename.split("-")

        primary_speaker = fileinfo[2].split("_")[-1][:-2] #fileinfo[2] = final2_primary_M1.T
        speakers = fileinfo[0]
        l_speaker,r_speaker = speakers.split("_")
        if primary_speaker == l_speaker:
            secondary_speaker = r_speaker
        else:
            secondary_speaker = l_speaker

        #print (primary_speaker,l_speaker,r_speaker,speaker_on_focus)

        file_df = pd.read_csv(each_file,sep="\t")
        file_df.columns = [el.replace(primary_speaker, "PRIMA").replace(secondary_speaker, "SECON") for el in file_df.columns]

        if primary_speaker != speaker_on_focus:
            #FILES TO TRAIN!
            train_files.append(file_df)
        else:
            #FILES TO TEST!
            test_files.append(file_df)
            
        
        
    training_data = pd.concat(train_files, ignore_index=True)
    testing_data = pd.concat(test_files, ignore_index=True)
    print ("Speaker on focus:",speaker_on_focus)
    print ("Training data size:",training_data.shape)
    print ("Testing data size:",testing_data.shape)
    print ()
    
    
    
    
    
    
    
    #Train data
    features=['PRIMA:velocity-r', 'PRIMA:velocity-clock', 'PRIMA:velocity-x', 'PRIMA:velocity-y',
              'PRIMA:acceleration-r', 'PRIMA:acceleration-clock', 'PRIMA:acceleration-x', 'PRIMA:acceleration-y'
             ]

    feats_learning = training_data[features]
    X = feats_learning.dropna() #REMEMBER THIS!!!! We're ignoring any row that has a NaN value.
    scaler = MinMaxScaler((0,1))
    scaler.fit(X)
    X_norm = scaler.transform(X)
    #X_norm = X
    #We're ignoring anyrow that has a NaN value.
    #Y = (training_data[(feats_learning.isnull().sum(axis=1)==0)]['PRIMA:HeadMovement'].equals('nod')*1).values #0: no movement; 1: movement
    Y = training_data[(feats_learning.isnull().sum(axis=1)==0)]['PRIMA:HeadMovement'].map(map_movements)
    
    #Test data
    feats_testing = testing_data[features]

    X_test = feats_testing.dropna() #REMEMBER THIS!!!! We're ignoring any row that has a NaN value.
    X_test_norm = scaler.transform(X_test)
    #X_test_norm = X_test
#    Y_test = (testing_data[(feats_testing.isnull().sum(axis=1)==0)]['PRIMA:HeadMovement'].equals('nod')*1).values #0: no movement; 1: movement
    Y_test =  testing_data[(feats_testing.isnull().sum(axis=1)==0)]['PRIMA:HeadMovement'].map(map_movements)
    
    classifiers_to_test = [(DummyClassifier(strategy="most_frequent"),"MF"),
                           (LogisticRegression(solver="liblinear"),"LR"),
                          (LinearSVC(),"LINEARSVC"),
                          (MLPClassifier(hidden_layer_sizes=(30,30,30,30), random_state=442), "MLP")
                          ]
                        #I think that now we have to fine-tune the n_hidden_layers and their sizes (MLP)
    
    
    for clf, clfname in classifiers_to_test:
        print ("Trying classifier ",clfname,speaker_on_focus)
        clf.fit(X_norm, Y)
        y_pred = clf.predict(X_test_norm)
        accuracy_results.loc[clfname,speaker_on_focus] = accuracy_score(Y_test,y_pred)
        #f1score_resultsb.loc[clfname,speaker_on_focus] = f1_score(Y_test,y_pred)
        f1score_results.loc[clfname,speaker_on_focus] = f1_score(Y_test,y_pred, average='macro')
        f1score_resultsw.loc[clfname,speaker_on_focus] = f1_score(Y_test,y_pred, average='weighted')
    
        print (confusion_matrix(Y_test,y_pred))
        print (accuracy_score(Y_test,y_pred)) #same as f1-score (micro-averagaed)
        print (f1_score(Y_test,y_pred, average='macro'))
        print (f1_score(Y_test,y_pred, average='weighted'))
        y_pred=None
        print ()

        
accuracy_results

Speaker on focus: F5
Training data size: (181647, 28)
Testing data size: (16413, 28)

Trying classifier  MF F5


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[[12154     0     0     0]
 [ 1330     0     0     0]
 [  914     0     0     0]
 [ 2015     0     0     0]]
0.7405105708889295
0.21272797283578954
0.6301092504347008

Trying classifier  LR F5


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[[11842     0     0   312]
 [ 1201     1     0   128]
 [  807     3     0   104]
 [ 1692     1     0   322]]
0.7411807713397917
0.2700432898347372
0.6608056259841936

Trying classifier  LINEARSVC F5


...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[[8682    1    0  633]
 [1655    5    0  140]
 [1581    1    0   89]
 [2944    7    0  794]]
0.5734938301475926
0.25442646283907067
0.47190485592994247

Trying classifier  LINEARSVC M6


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[[8743    0    0  573]
 [1673    1    0  126]
 [1589    0    0   82]
 [3056    1    0  688]]
0.5705298814420517
0.24558256293274908
0.4641200546578269

Trying classifier  MLP M6
[[8272  111    8  925]
 [1463  127    0  210]
 [1369   32    5  265]
 [2020  144    5 1576]]
0.6036777159448342
0.33171946908328764
0.5347801920921669



,F5,F6,M2,M4,F1,F3,M3,M5,M1,F2,F4,M6
MF,0.740511,0.506497,0.714565,0.648316,0.671644,0.588701,0.719731,0.551088,0.692986,0.626404,0.591385,0.563513
LR,0.741181,0.551450,0.727142,0.651950,0.701877,0.595710,0.719851,0.583654,0.716773,0.659744,0.629262,0.573494
LINEARSVC,0.741364,0.547775,0.725448,0.651587,0.698222,0.595100,0.721354,0.579461,0.715997,0.658529,0.625657,0.570530
MLP,0.726985,0.574025,0.740046,0.658975,0.730252,0.620818,0.714019,0.611280,0.722318,0.655068,0.656326,0.603678


In [44]:
accuracy_results

,F5,F6,M2,M4,F1,F3,M3,M5,M1,F2,F4,M6
MF,0.740511,0.506497,0.714565,0.648316,0.671644,0.588701,0.719731,0.551088,0.692986,0.626404,0.591385,0.563513
LR,0.741181,0.551450,0.727142,0.651950,0.701877,0.595710,0.719851,0.583654,0.716773,0.659744,0.629262,0.573494
LINEARSVC,0.741364,0.547775,0.725448,0.651587,0.698222,0.595100,0.721354,0.579461,0.715997,0.658529,0.625657,0.570530
MLP,0.726985,0.574025,0.740046,0.658975,0.730252,0.620818,0.714019,0.611280,0.722318,0.655068,0.656326,0.603678


In [45]:
f1score_resultsw

,F5,F6,M2,M4,F1,F3,M3,M5,M1,F2,F4,M6
MF,0.630109,0.340577,0.595607,0.509992,0.539715,0.436293,0.602434,0.391594,0.567316,0.482515,0.439536,0.406197
LR,0.660806,0.457568,0.635905,0.558570,0.623474,0.472865,0.646007,0.467112,0.671706,0.579740,0.530091,0.471905
LINEARSVC,0.657262,0.450289,0.630643,0.554351,0.614610,0.467695,0.642290,0.459122,0.667810,0.573500,0.523899,0.464120
MLP,0.679067,0.548749,0.681271,0.609796,0.692442,0.544676,0.684146,0.522273,0.689109,0.625757,0.602174,0.534780


In [46]:
f1score_results

,F5,F6,M2,M4,F1,F3,M3,M5,M1,F2,F4,M6
MF,0.212728,0.168104,0.208381,0.196660,0.200893,0.185278,0.209257,0.177646,0.204664,0.192573,0.185808,0.180207
LR,0.270043,0.275461,0.261201,0.251415,0.283873,0.224611,0.266704,0.246481,0.313658,0.296592,0.299665,0.254426
LINEARSVC,0.262817,0.269165,0.254061,0.246437,0.274042,0.218731,0.259514,0.239027,0.309384,0.288442,0.293392,0.245583
MLP,0.328970,0.384606,0.333174,0.335292,0.401110,0.306929,0.356594,0.306922,0.361703,0.391476,0.386255,0.331719


In [47]:
accuracy_results.T.describe()

,MF,LR,LINEARSVC,MLP
count,12.000000,12.000000,12.000000,12.000000
mean,0.634612,0.654341,0.652585,0.667816
std,0.075262,0.067090,0.068194,0.057532
min,0.506497,0.551450,0.547775,0.574025
25%,0.582404,0.592696,0.591191,0.618433
50%,0.637360,0.655847,0.655058,0.657651
75%,0.698381,0.717542,0.717336,0.723484
max,0.740511,0.741181,0.741364,0.740046


In [48]:
f1score_results.T.describe()

,MF,LR,LINEARSVC,MLP
count,12.000000,12.000000,12.000000,12.000000
mean,0.193517,0.270344,0.263383,0.352063
std,0.014236,0.025233,0.025379,0.032975
min,0.168104,0.224611,0.218731,0.306922
25%,0.184010,0.253674,0.246223,0.331032
50%,0.194617,0.268374,0.261166,0.345943
75%,0.205593,0.287053,0.277642,0.385019
max,0.212728,0.313658,0.309384,0.401110
